In [1]:
import sys
sys.path.append('../code')

In [2]:
import json
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [11]:
from make_patch_df import *
from patch_nlp import get_info_from_df, process_numbers

In [4]:
hero_name_set = get_hero_names()
ability_set = get_abi_names('../../dota2-database/json/abilities.json')

In [5]:
patch_url_dict = {}
patch_url_dict['6.86'] = 'December_16,_2015_Patch'
patch_url_dict['6.85'] = 'September_24,_2015_Patch'
patch_url_dict['6.84'] = 'April_30,_2015_Patch'
patch_url_dict['6.83'] = 'December_17,_2014_Patch'
patch_url_dict['6.82'] = 'September_25,_2014_Patch'
patch_url_dict['6.81'] = 'April_29,_2014_Patch'
patch_url_dict['6.80'] = 'January_29,_2014_Patch'
patch_url_dict['6.79'] = 'October_21,_2013_Patch'
patch_url_dict['6.78'] = 'June_04,_2013_Patch'
patch_url_dict['6.77'] = 'December_19,_2012_Patch'

In [6]:
with open('../data/patch_685_df.pkl') as f:
    df_685_full = pickle.load(f)
    
# df_685 = df_from_patch_query('September_24,_2015_Patch', hero_name_set, '6.85')

In [7]:
df_685_full.head(6)

,hero,patch,text,num_buffs,num_nerfs,change_type,text_no_abi
0,Abaddon,6.85,"[Mist Coil cooldown reduced from 5 to 4.5, Bor...",2,0,"[1, 1]","[cooldown reduced from 5 to 4.5, Aghanim's Sce..."
1,Alchemist,6.85,[Acid Spray damage increased from 12/16/20/24 ...,2,0,"[1, 1]",[damage increased from 12/16/20/24 to 15/20/25...
2,Ancient Apparition,6.85,[Cold Feet cast range increased from 700 to 70...,1,0,[1],[cast range increased from 700 to 700/800/900/...
3,Anti-Mage,6.85,[Blink distance rescaled from 1000/1075/1150/1...,0,2,"[-1, -1]",[distance rescaled from 1000/1075/1150/1150 to...
4,Arc Warden,6.85,[Spark Wraith cooldown reduced from 7 to 4],1,0,[1],[Spark Wraith cooldown reduced from 7 to 4]
5,Axe,6.85,[],0,0,[],[]


In [8]:
texts_685, labels_685, ratios_685 = get_info_from_df(df_685_full, labeled=True)

In [9]:
df_684 = df_from_patch_query('April_30,_2015_Patch', hero_name_set, '6.84')
df_684.head(4)
df_684['text_no_abi'] = df_684['text'].apply(remove_abi_names, abi_set=ability_set)

In [10]:
texts_684, labels_684, ratios_684 = get_info_from_df(df_684, labeled=False)

In [12]:
df_684['text_no_abi'][19]

[u':\n\n Movement and attack slow from -30 to -20/30/40/50\n\n Slow duration from 3.5/4/4.5/5 to 4.5',
 u'cooldown reduced from 10/9/8/7 to 9/8/7/6',
 u'cooldown from 150/120/90 to 90']

In [16]:
tfidf_train = TfidfVectorizer(use_idf=False)
tfmat_train_685 = tfidf_train.fit_transform(texts_685)
tfmat_test_684 = tfidf_train.transform(texts_684)

gdb_685 = GradientBoostingClassifier(random_state=0)
gdb_685.fit(tfmat_train_685.toarray(), labels_685)
gdb_685.predict_proba(tfmat_test_684.toarray())[:,1][:10]

array([ 0.92983907,  0.84364172,  0.7527597 ,  0.94395809,  0.51693776,
        0.95258947,  0.84770223,  0.66231943,  0.84966407,  0.23502058])

In [17]:
model_output = np.round(gdb_685.predict_proba(tfmat_test_684.toarray())[:,1], 4)
counter = 0
predictions = []
ratios_by_hero = []
for item in df_684['text']:
    predictions.append(model_output[counter:counter+len(item)])
    ratios_by_hero.append(ratios_684[counter:counter+len(item)])
    counter += len(item)
df_684['predictions'] = predictions
df_684['ratios'] = ratios_by_hero

In [18]:
df_684.head(7)

,hero,patch,text,text_no_abi,predictions,ratios
0,Abaddon,6.84,[Borrowed Time can now be cast while disabled ...,[can now be cast while disabled (same rules as...,[0.9298],[None]
1,Alchemist,6.84,[Unstable Concoction:\n\n Countdown can now be...,[:\n\n Countdown can now be seen by enemies\n\...,"[0.8436, 0.7528, 0.944]","[None, None, None]"
2,Ancient Apparition,6.84,[Chilling Touch buff can now be removed by cli...,[buff can now be removed by clicking on the bu...,[0.5169],[None]
3,Anti-Mage,6.84,[],[],[],[]
4,Arc Warden,6.84,[],[],[],[]
5,Axe,6.84,"[Berserker's Call cast point from 0.3 to 0.4, ...","[cast point from 0.3 to 0.4, now triggers when...","[0.9526, 0.8477, 0.6623]","[1.333, None, 0.75]"
6,Bane,6.84,[Brain Sap mana cost from 125/150/175/200 to 1...,[mana cost from 125/150/175/200 to 100/125/150...,"[0.8497, 0.235, 0.8477]","[0.846, None, None]"


In [46]:
zip(df_684.predictions, df_684.text)

[(array([ 0.9248]),
  [u"Borrowed Time can now be cast while disabled (same rules as Morphling's Morph)"]),
 (array([ 0.8011,  0.7333,  0.942 ]),
  [u'Unstable Concoction:\n\n Countdown can now be seen by enemies\n\n Now has an area targeting cursor',
   u"Greevil's Greed now grants 4x gold bounty from Runes",
   u"Alchemist can now cast  Aghanim's Scepter to directly grant any allied hero all Aghanim's Scepter bonuses as a buff (the hero upgrade and the stat upgrade). The scepter is consumed in the process. Multiple instances of this buff do not stack. Alchemist can target himself as well"]),
 (array([ 0.6352]),
  [u'Chilling Touch buff can now be removed by clicking on the buff icon above the health bar']),
 (array([], dtype=float64), []),
 (array([], dtype=float64), []),
 (array([ 0.9509,  0.8303,  0.6542]),
  [u"Berserker's Call cast point from 0.3 to 0.4",
   u"Counter Helix now triggers when attacks land rather than when they start\n\n Affects  Centaur Warrunner's  Return and  Ti

In [59]:
df_684.text[1]

[u'Unstable Concoction:\n\n Countdown can now be seen by enemies\n\n Now has an area targeting cursor',
 u"Greevil's Greed now grants 4x gold bounty from Runes",
 u"Alchemist can now cast  Aghanim's Scepter to directly grant any allied hero all Aghanim's Scepter bonuses as a buff (the hero upgrade and the stat upgrade). The scepter is consumed in the process. Multiple instances of this buff do not stack. Alchemist can target himself as well"]